
# Tables 1, 2, and 3

In [3]:
from functools import partial
from multiprocess import Pool, cpu_count, current_process
import pandas as pd
from tqdm.notebook import tqdm

from pref_voting.generate_weighted_majority_graphs import *
from pref_voting.voting_methods import *

from pref_voting.generate_profiles import *

# needed to ensure that random numbers are different in each process (for multiprocessing)
from os import register_at_fork
register_at_fork(after_in_child=np.random.seed)


In [4]:
## Condorcetian Candidates

def is_condorcetian(vm, cand, edata): 
    """
    Definition 4.4: Return True if cand is Condorcetian in edata according to vm
    """
    candidates = edata.candidates
    
    for c in edata.dominates(cand): # cand is majority preferred to c
        ws_minus = vm(edata, curr_cands = [_c for _c in candidates if _c != c])
        if cand in ws_minus: 
            return True
    return False

        
def is_weak_condorcetian(vm, cand, edata): 
    """
    Definition 4.4: Return True if cand is weak Condorcetian in edata according to vm
    """
    
    candidates = edata.candidates
    
    for c in edata.candidates:
        if edata.majority_prefers(cand, c) or edata.is_tied(cand, c):
            ws_minus = vm(edata, curr_cands = [_c for _c in candidates if _c != c])
            if cand in ws_minus: 
                return True        
    return False



## Table 1

Of four-candidate linear profiles in which $BP(\mathbf{P}) \neq SC(\mathbf{P})$, find the percentage in which Beat Path violates strong stability for winners, i.e., there are $a,b\in X(\mathbf{P})$ such that $a\in BP(\mathbf{P}_{-b})$, $Margin_\mathbf{P}(a, b) \geq 0$, and $a\notin BP(\mathbf{P})$

In [5]:
df = pd.read_csv("./tables-data-FINAL/violations_ssw.csv")

In [6]:
df

,vm,comparison_vm,num_cands,num_voters,num_profiles,probmod,perc_diff_ws,perc_ssw_violation,perc_ssw_violation_cond_diff_ws
0,Beat Path,Split Cycle,4,"(10, 11)",2000000,IC,0.005159,0.005148,0.997868
1,Beat Path,Split Cycle,4,"(20, 21)",2000000,IC,0.008280,0.008157,0.985085
2,Beat Path,Split Cycle,4,"(50, 51)",2000000,IC,0.010549,0.010040,0.951749
3,Beat Path,Split Cycle,4,"(100, 101)",2000000,IC,0.011314,0.010560,0.933316
4,Beat Path,Split Cycle,4,"(500, 501)",2000000,IC,0.012127,0.010996,0.906778
5,Beat Path,Split Cycle,4,"(1000, 1001)",2000000,IC,0.012480,0.011239,0.900521
6,Beat Path,Split Cycle,4,"(5000, 5001)",2000000,IC,0.012315,0.011045,0.896914
7,Beat Path,Split Cycle,4,"(10, 11)",2000000,IAC,0.005325,0.005313,0.997746
8,Beat Path,Split Cycle,4,"(20, 21)",2000000,IAC,0.008195,0.007975,0.973034
9,Beat Path,Split Cycle,4,"(50, 51)",2000000,IAC,0.009745,0.009069,0.930631


In [7]:
print("\nOf the profiles in which the Beat Path and Split Cycle winners are different, the percent of strong stability for winners violations for Beat Path (Table 1).")
df_table = df.pivot(index="probmod", columns="num_voters", values="perc_ssw_violation_cond_diff_ws")
df_table


Of the profiles in which the Beat Path and Split Cycle winners are different, the percent of strong stability for winners violations for Beat Path (Table 1).


num_voters,"(10, 11)","(100, 101)","(1000, 1001)","(20, 21)","(50, 51)","(500, 501)","(5000, 5001)"
probmod,,,,,,,
IAC,0.997746,0.912119,0.897506,0.973034,0.930631,0.899583,0.895494
IC,0.997868,0.933316,0.900521,0.985085,0.951749,0.906778,0.896914
MALLOWS-RELPHI-R,0.998290,0.941248,0.889693,0.989275,0.966455,0.910811,0.895683
URN-R,0.991946,0.918287,0.899818,0.969054,0.926678,0.907615,0.897029


In [8]:
print("\nThe percent of strong stability for winners violations for Beat Path.")

df_table = df.pivot(index="probmod", columns="num_voters", values="perc_ssw_violation")
df_table


The percent of strong stability for winners violations for Beat Path.


num_voters,"(10, 11)","(100, 101)","(1000, 1001)","(20, 21)","(50, 51)","(500, 501)","(5000, 5001)"
probmod,,,,,,,
IAC,0.005313,0.009502,0.009680,0.007975,0.009069,0.009595,0.009588
IC,0.005148,0.010560,0.011239,0.008157,0.010040,0.010996,0.011045
MALLOWS-RELPHI-R,0.001167,0.000777,0.000246,0.001337,0.001066,0.000337,0.000124
URN-R,0.002217,0.003827,0.003961,0.003241,0.003652,0.003886,0.003925


In [9]:
print("\nThe percent of profiles in which Beat Path and Split Cycle have different winners.")
df_table = df.pivot(index="probmod", columns="num_voters", values="perc_diff_ws")
df_table


The percent of profiles in which Beat Path and Split Cycle have different winners.


num_voters,"(10, 11)","(100, 101)","(1000, 1001)","(20, 21)","(50, 51)","(500, 501)","(5000, 5001)"
probmod,,,,,,,
IAC,0.005325,0.010417,0.010786,0.008195,0.009745,0.010665,0.010708
IC,0.005159,0.011314,0.012480,0.008280,0.010549,0.012127,0.012315
MALLOWS-RELPHI-R,0.001169,0.000825,0.000276,0.001352,0.001103,0.000370,0.000139
URN-R,0.002235,0.004167,0.004402,0.003344,0.003941,0.004281,0.004375


In [10]:

def has_strong_stability_for_winners_violation(vm, edata, weak_condorcetian_candidates = None): 
    """
    Definition 4.2.3: Return True if there are candidates a and b such that
      1. a in a winner according to vm in edata without b in the election, 
      2. a is majority preferred to b or tied with b, and 
      3. a is not a winner in edata
    
    This is equivalent to requiring that all weak Condorcetian candidates are winners in edata according to vm. 
    """
    candidates = edata.candidates
    
    if weak_condorcetian_candidates is None: 
        weak_condorcetian_candidates = [c for c in candidates if is_weak_condorcetian(vm, c, edata)]
    
    ws = vm(edata)
        
    return not set(weak_condorcetian_candidates).issubset(set(ws))



In [11]:
def record_strong_stability_for_winners_violation(vms, main_vm, num_cands, num_voters, probmod,  t): 
    
    prof = generate_profile(num_cands, num_voters, probmod=probmod)
    
    data = {vm.name: dict() for vm in vms}
    for vm in vms: 
        weak_condorcetian_candidates = [c for c in prof.candidates if is_weak_condorcetian(vm, c, prof)]
        
        main_ws = main_vm(prof)
        vm_ws = vm(prof)
        data[vm.name] =  {
            "diff_ws": main_ws != vm_ws, 
            "ssw_violation": has_strong_stability_for_winners_violation(vm, prof, weak_condorcetian_candidates = weak_condorcetian_candidates)
        }
    return data
    

In [12]:
## Parameters for the simulation

# WARNING: The simulation takes a very long time!  
# Set the following to True to skip running the simulation
SKIP_SIMULATION = True  

num_trials = 1_000_000 # WARNING: This takes a very long time to run!

numbers_of_candidates = [
#     3,
    4,
#     5,
#     6,
#     7,
#     8,
#     10, 
#     15, 
#     20
]

numbers_of_voters = [
    (10, 11),
    (20, 21),
    (50, 51),
    (100, 101),
    (500, 501),
    (1000, 1001),
    (5000, 5001),
] 

# uncomment probability models as needed
prob_models = [
    "IC", 
    "IAC", 
    "MALLOWS-RELPHI-R", 
    "URN-R"
]
 
vms = [
    beat_path,
#     llull,
#     copeland,
#     minimax,
#     uc_gill, # uncovered set
#     instant_runoff,  
#     borda,
#     plurality,
#     getcha
]

main_vm = split_cycle

# use parallel processing to speed up the search
cpus = cpu_count()
print(f'CPUS: {cpus}')
pool = Pool(10) # set to cpus when you can use all the processors

print(f"SKIP_SIMULATION is set to {SKIP_SIMULATION}")

CPUS: 12
SKIP_SIMULATION is set to True


In [13]:
%%time 

if not SKIP_SIMULATION:
    data_for_df = {
        "vm": list(),
        "comparison_vm": list(),
        "num_cands": list(), 
        "num_voters": list(),
        "num_profiles": list(),
        "probmod": list(),
        "perc_diff_ws": list(),
        "perc_ssw_violation": list(),
        "perc_ssw_violation_cond_diff_ws": list()
    }

    for probmod in prob_models:

        print(probmod)

        voting_scenarios = list(product(numbers_of_candidates, numbers_of_voters))

        data = {vm.name: {_vs: {"total_profiles": 0, 
                                "total_even": list(),
                                "total_odd": list()} 
                        for _vs in voting_scenarios} 
                for vm in vms}

        for vs in voting_scenarios:
            print(f"     {vs}")
            num_candidates = vs[0]
            num_voters_even = vs[1][0]
            num_voters_odd = vs[1][1]

            # find the violations in parallel

            record_violations_even = partial(
                record_strong_stability_for_winners_violation,
                vms, 
                main_vm, 
                num_candidates, 
                num_voters_even, 
                probmod)

            record_violations_odd = partial(
                record_strong_stability_for_winners_violation,
                vms, 
                main_vm, 
                num_candidates, 
                num_voters_odd, 
                probmod)

            violations_even = pool.map(record_violations_even, range(num_trials))
            violations_odd = pool.map(record_violations_odd, range(num_trials))

            # done calculating the violations
            for vm in vms:
                data_even = [d[vm.name] for d in violations_even]
                data_odd = [d[vm.name] for d in violations_odd]
                all_data = data_even + data_odd

                num_profiles = len(all_data)

                data_for_df["vm"].append(vm.name)
                data_for_df["comparison_vm"].append(main_vm.name)
                data_for_df["num_cands"].append(num_candidates)
                data_for_df["num_voters"].append(str((num_voters_even, num_voters_odd)))
                data_for_df["probmod"].append(probmod)            
                data_for_df["num_profiles"].append(num_profiles)

                data_for_df["perc_diff_ws"].append(sum([d["diff_ws"] for d in all_data]) / num_profiles)
                data_for_df["perc_ssw_violation"].append(sum([d["ssw_violation"] for d in all_data]) / num_profiles)
                
                data_with_diff_ws = [d for d in all_data if d["diff_ws"]]
                
                if len(data_with_diff_ws) > 0: 
                    data_for_df["perc_ssw_violation_cond_diff_ws"].append(sum([d["ssw_violation"] for d in data_with_diff_ws]) / len(data_with_diff_ws))
                else: 
                    data_for_df["perc_ssw_violation_cond_diff_ws"].append(0.0)
                    
            df = pd.DataFrame(data_for_df)
            df.to_csv(f"./violations_ssw.csv",index=False)
            


    print("done.")



CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 3.1 µs


## Table 2

Of four-candidate linear profiles in which $BP(\mathbf{P})\neq SV(\mathbf{P})$ find the percentage in which Beat Path violates strong stability for winners with tiebreaking.

In [14]:
df = pd.read_csv("./tables-data-FINAL/violations_ssw_w_tb.csv")

In [15]:
print("\nOf the profiles in which the Beat Path and Stable Voting winners are different, ")
print("the percent of strong stability for winners with tiebreaking violations for Beat Path (Table 2).")
df_table = df.pivot(index="probmod", columns="num_voters", values="perc_ssw_w_tb_violation_cond_diff_ws")
df_table


Of the profiles in which the Beat Path and Stable Voting winners are different, 
the percent of strong stability for winners with tiebreaking violations for Beat Path (Table 2).


num_voters,"(10, 11)","(100, 101)","(1000, 1001)","(20, 21)","(50, 51)","(500, 501)","(5000, 5001)"
probmod,,,,,,,
IAC,0.275862,0.333333,0.0,0.176471,0.230769,0.333333,1.0
IC,0.129032,0.000000,0.0,0.428571,0.000000,1.000000,0.0
MALLOWS-RELPHI-R,0.222222,0.000000,0.0,0.400000,0.000000,0.000000,0.0
URN-R,0.260870,0.428571,0.0,0.142857,0.222222,0.500000,0.0


In [16]:
print("\nThe percent of strong stability for winners with tiebreaking violations for Beat Path.")
df_table = df.pivot(index="probmod", columns="num_voters", values="perc_ssw_w_tb_violation")
df_table


The percent of strong stability for winners with tiebreaking violations for Beat Path.


num_voters,"(10, 11)","(100, 101)","(1000, 1001)","(20, 21)","(50, 51)","(500, 501)","(5000, 5001)"
probmod,,,,,,,
IAC,0.08,0.020,0.0,0.030,0.015,0.005,0.01
IC,0.06,0.000,0.0,0.045,0.000,0.045,0.00
MALLOWS-RELPHI-R,0.01,0.000,0.0,0.010,0.000,0.000,0.00
URN-R,0.03,0.015,0.0,0.010,0.010,0.005,0.00


In [17]:
print("\nThe percent of profiles in which Beat Path and Stable Voting have different winners.")
df_table = df.pivot(index="probmod", columns="num_voters", values="perc_diff_ws")
df_table


The percent of profiles in which Beat Path and Stable Voting have different winners.


num_voters,"(10, 11)","(100, 101)","(1000, 1001)","(20, 21)","(50, 51)","(500, 501)","(5000, 5001)"
probmod,,,,,,,
IAC,0.290,0.060,0.010,0.170,0.065,0.015,0.01
IC,0.465,0.105,0.045,0.105,0.105,0.045,0.00
MALLOWS-RELPHI-R,0.045,0.000,0.000,0.025,0.010,0.000,0.00
URN-R,0.115,0.035,0.000,0.070,0.045,0.010,0.00


In [18]:


def has_strong_stability_for_winners_with_tiebreaking_violation(vm, edata, condorcetian_candidates = None, weak_condorcetian_candidates = None): 

    """
    Definition 4.20: Return True if there is a vioalation of strong stability winners with tiebreaking.  
    This function returns True when: 
       
       1.  vm vioaltes stability for winners with tiebreaking for edata, or
       2.  there some candidate is weakly Condorcetian for vm in edata and no candidate is Condorcetian for vm in edata, and
           it is not the case that all candidates who win in edata according to vm are weakly Condorcetian for vm in edata.

    """
    candidates = edata.candidates
    
    if condorcetian_candidates is None: 
        condorcetian_candidates = [c for c in candidates if is_condorcetian(vm, c, edata)]
        
    if weak_condorcetian_candidates is None: 
        weak_condorcetian_candidates = [c for c in candidates if is_weak_condorcetian(vm, c, edata)]
        
    ws = vm(edata)
        
    return (len(condorcetian_candidates) > 0 and not set(ws).issubset(set(condorcetian_candidates)))\
    or (len(weak_condorcetian_candidates) != 0 and len(condorcetian_candidates) == 0 and not set(ws).issubset(set(weak_condorcetian_candidates)))





In [19]:
def record_strong_stability_for_winners_with_tiebreaking_violation(
    vms, 
    main_vm, 
    num_cands, 
    num_voters, 
    probmod, 
    t): 
    
    np.random.seed(current_process().pid)
    prof = generate_profile(num_cands, num_voters, probmod=probmod)
    
    data = {vm.name: dict() for vm in vms}
    for vm in vms: 
        condorcetian_candidates = [c for c in prof.candidates if is_condorcetian(vm, c, prof)]
        weak_condorcetian_candidates = [c for c in prof.candidates if is_weak_condorcetian(vm, c, prof)]
        
        main_ws = main_vm(prof)
        vm_ws = vm(prof)
        data[vm.name] =  {
            "diff_ws": main_ws != vm_ws, 
            "ssw_w_tb_violation": has_strong_stability_for_winners_with_tiebreaking_violation(vm, 
                                                                                               prof, 
                                                                                               condorcetian_candidates = condorcetian_candidates,
                                                                                               weak_condorcetian_candidates = weak_condorcetian_candidates)
        }
    return data


In [20]:
## Parameters for the simulation

# WARNING: The simulation takes a very long time!  
# Set the following to True to skip running the simulation
SKIP_SIMULATION = True  

num_trials = 100#_000

numbers_of_candidates = [
#     3,
    4,
#     5,
#     6,
#     7,
#     8,
#     10, 
#     15, 
#     20
]

numbers_of_voters = [
    (10, 11),
    (20, 21),
    (50, 51),
    (100, 101),
    (500, 501),
    (1000, 1001),
    (5000, 5001),
] 

# uncomment probability models as needed
prob_models = [
    "IC", 
    "IAC", 
    "MALLOWS-RELPHI-R", 
    "URN-R"
]
 
vms = [
    beat_path,
#     llull,
#     copeland,
#     minimax,
#     uc_gill, # uncovered set
#     instant_runoff,  
#     borda,
#     plurality,
#     getcha
]

main_vm = stable_voting

# use parallel processing to speed up the search
cpus = cpu_count()
print(f'CPUS: {cpus}')
pool = Pool(10) # set to cpus when you can use all the processors

print(f"SKIP_SIMULATION is set to {SKIP_SIMULATION}")

CPUS: 12
SKIP_SIMULATION is set to True


In [21]:
%%time 

if not SKIP_SIMULATION:
    data_for_df = {
        "vm": list(),
        "comparison_vm": list(),
        "num_cands": list(), 
        "num_voters": list(),
        "num_profiles": list(),
        "probmod": list(),
        "perc_diff_ws": list(),
        "perc_ssw_w_tb_violation": list(),
        "perc_ssw_w_tb_violation_cond_diff_ws": list()
    }

    for probmod in prob_models:

        print(probmod)

        voting_scenarios = list(product(numbers_of_candidates, numbers_of_voters))

        data = {vm.name: {_vs: {"total_profiles": 0, 
                                "total_even": list(),
                                "total_odd": list()} 
                        for _vs in voting_scenarios} 
                for vm in vms}

        for vs in voting_scenarios:
            print(f"     {vs}")
            num_candidates = vs[0]
            num_voters_even = vs[1][0]
            num_voters_odd = vs[1][1]

            # find the violations in parallel

            record_violations_even = partial(
                record_strong_stability_for_winners_with_tiebreaking_violation,
                vms, 
                main_vm, 
                num_candidates, 
                num_voters_even, 
                probmod)

            record_violations_odd = partial(
                record_strong_stability_for_winners_with_tiebreaking_violation,
                vms, 
                main_vm, 
                num_candidates, 
                num_voters_odd, 
                probmod)

            violations_even = pool.map(record_violations_even, range(num_trials))
            violations_odd = pool.map(record_violations_odd, range(num_trials))

            # done calculating the violations
            for vm in vms:
                data_even = [d[vm.name] for d in violations_even]
                data_odd = [d[vm.name] for d in violations_odd]
                all_data = data_even + data_odd
                num_profiles = len(all_data)

                data_for_df["vm"].append(vm.name)
                data_for_df["comparison_vm"].append(main_vm.name)
                data_for_df["num_cands"].append(num_candidates)
                data_for_df["num_voters"].append(str((num_voters_even, num_voters_odd)))
                data_for_df["num_profiles"].append(num_profiles)
                data_for_df["probmod"].append(probmod)
                data_for_df["perc_diff_ws"].append(sum([d["diff_ws"] for d in all_data]) / num_profiles)
                data_for_df["perc_ssw_w_tb_violation"].append(sum([d["ssw_w_tb_violation"] for d in all_data]) / num_profiles)
                
                data_with_diff_ws = [d for d in all_data if d["diff_ws"]]
                
                if len(data_with_diff_ws) > 0: 
                    data_for_df["perc_ssw_w_tb_violation_cond_diff_ws"].append(sum([d["ssw_w_tb_violation"] for d in data_with_diff_ws]) / len(data_with_diff_ws))
                else: 
                    data_for_df["perc_ssw_w_tb_violation_cond_diff_ws"].append(0.0)
                    
            df = pd.DataFrame(data_for_df)
            df.to_csv(f"./violations_ssw_w_tb.csv",index=False)
            
    print("done.")


CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 3.1 µs


## Table 3

Estimated average sizes of winning sets for profiles with a given number of candidates in the limit as the number of voters goes to infinity.

In [22]:
df = pd.read_csv("./tables-data-FINAL/irresoluteness_inf_voters_ADDITIONAL_TEST2.csv")


In [23]:
print("\n Average Winning Set Size (Table 3)\n")
df_table = df.pivot(index="vm", columns="num_cands", values="avg_ws_size")
df_table


 Average Winning Set Size (Table 3)



num_cands,3,4,5,6,7,8,9,10,20,30
vm,,,,,,,,,,
Copeland,1.175648,1.260324,1.288842,1.301580,1.309300,1.312284,1.312722,1.311334,1.278053,1.247802
GETCHA,1.175648,1.440927,1.786059,2.210534,2.715445,3.299664,3.956272,4.678948,13.524865,22.914119
Split Cycle,1.000000,1.012480,1.032526,1.056392,1.082383,1.109546,1.136609,1.164322,1.417049,1.624962
Uncovered Set,1.175648,1.350626,1.529297,1.710961,1.896472,2.084839,2.274966,2.468487,4.550156,6.840625


In [24]:
print("\n Proportion with Multiple Winners\n")
df_table = df.pivot(index="vm", columns="num_cands", values="perc_mult_winners")
df_table


 Proportion with Multiple Winners



num_cands,3,4,5,6,7,8,9,10,20,30
vm,,,,,,,,,,
Copeland,0.087824,0.175313,0.217052,0.234268,0.242372,0.246132,0.247617,0.247456,0.227518,0.206849
GETCHA,0.087824,0.175313,0.251198,0.315078,0.369246,0.415267,0.454564,0.488688,0.681362,0.764981
Split Cycle,0.000000,0.012480,0.031932,0.054349,0.077936,0.101794,0.124684,0.147368,0.322293,0.432342
Uncovered Set,0.087824,0.175313,0.251198,0.315078,0.369246,0.415267,0.454564,0.488688,0.681362,0.764981


In [25]:
print("\n Proportion of Profiles with a Condorcet Winner\n")
df_table = df.pivot(index="vm", columns="num_cands", values="perc_condorcet_winner")
df_table


 Proportion of Profiles with a Condorcet Winner



num_cands,3,4,5,6,7,8,9,10,20,30
vm,,,,,,,,,,
Copeland,0.912176,0.824687,0.748802,0.684922,0.630754,0.584733,0.545436,0.511312,0.318638,0.235018
GETCHA,0.912176,0.824687,0.748802,0.684922,0.630754,0.584733,0.545436,0.511312,0.318638,0.235018
Split Cycle,0.912176,0.824687,0.748802,0.684922,0.630754,0.584733,0.545436,0.511312,0.318638,0.235018
Uncovered Set,0.912176,0.824687,0.748802,0.684922,0.630754,0.584733,0.545436,0.511312,0.318638,0.235018


In [26]:
all_num_cands = [
        3,
        4,
        5,
        6,
        7,
        8,
        9, 
        10,
        20,
        30
    ]

vms = [
    split_cycle, 
    copeland, 
    uc_gill,
    getcha
]
num_cpus = 10
num_trials = 5

In [27]:
def record_ws_data(vms, num_cand, t): 
    np.random.seed(current_process().pid)
    mg = generate_edge_ordered_tournament_infinite_limit(num_cand)
    winning_sets = {vm.name: vm(mg) for vm in vms}
    return {
        "has_cw": mg.condorcet_winner() is not None,
        "winning_set_sizes": {vm.name: len(winning_sets[vm.name]) for vm in vms}
        }

def find_irresoluteness_data_inf_voters(
    vms, 
    num_trials = 10, 
    all_num_cands = [
        3,
        4,
        5,
        6,
        7,
        8,
        9, 
        10,
        20,
        30
    ],
    num_cpus = 10
):
    data_for_df = {
        "num_cands": list(), 
        "num_margin_graphs": list(),
        "vm": list(),
        "perc_mult_winners": list(),
        "avg_ws_size": list(),
        "perc_condorcet_winner": list()
    }
    
    
    for num_cands in tqdm(all_num_cands): 
        print(num_cands)
        
        get_ws_data= partial(record_ws_data,
                             vms,
                             num_cands)

        pool = Pool(num_cpus) # set to cpus when you can use all the processors
        total_ws_data = pool.map(get_ws_data, range(num_trials))
        
        for vm in vms: 
            data = [d["winning_set_sizes"][vm.name] for d in total_ws_data]
            data_for_df["num_cands"].append(num_cands)
            data_for_df["num_margin_graphs"].append(len(data))
            data_for_df["vm"].append(vm.name)
            data_for_df["perc_mult_winners"].append(sum([d > 1 
                                                         for d in data]) / len(data) 
                                                    if len(data) > 0 else np.nan)
            data_for_df["avg_ws_size"].append(np.average([d for d in data]))
            data_for_df["perc_condorcet_winner"].append(sum([d["has_cw"]  for d in total_ws_data]) / len(total_ws_data))
            
        df = pd.DataFrame(data_for_df)
        df.to_csv(f"./irresoluteness_inf_voters.csv",index=False)
            
    
    print("done.")
    return df